In [1]:
import os
from PyPDF2 import PdfReader
import gensim
from gensim import corpora
from nltk.corpus import stopwords
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
from pprint import pprint

/home/keval/envs/topicmodeling/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


In [2]:
files = [f for f in os.listdir("pdfs/") if os.path.isfile("pdfs/"+f) and f.endswith('.pdf')]

data = []

for name in files:
    reader = PdfReader("pdfs/"+name)
    number_of_pages = len(reader.pages)
    text = ""
    for i in range(number_of_pages):
        page = reader.pages[i]
        tt = page.extract_text()
        text +=tt
    data.append(text)
    
print(data[0][:150])

Vifor Pharma Ltd. 
 2018 Responsibility Highlights Report
1 
RESPONSIBILITY  
HIGHLIGHTS REPORT
2018
Vifor Pharma Ltd. 
 CSR Report 2018
2
Vifor Pharm


In [4]:
def doc_to_words(doc):
    return(gensim.utils.simple_preprocess(str(doc), deacc=True))

for idx, item in enumerate(data):
    data[idx]=list(doc_to_words(item))
    
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'group', 'financial', 'report'])

def remove_stopwords(texts):
    return [word for word in texts if word not in stop_words]

for idx, item in enumerate(data):
    data[idx]=remove_stopwords(item)
print(data[0][:100])

['vifor', 'pharma', 'ltd', 'responsibility', 'highlights', 'responsibility', 'highlights', 'vifor', 'pharma', 'ltd', 'csr', 'vifor', 'pharma', 'ltd', 'responsibility', 'highlights', 'commitment', 'ifor', 'pharma', 'glance', 'ur', 'responsibility', 'approach', 'managing', 'esponsibilities', 'esponsibility', 'model', 'aterial', 'areas', 'hapter', 'page', 'business', 'behaviour', 'transparency', 'patient', 'needs', 'support', 'access', 'healthcare', 'employee', 'potential', 'workplace', 'ell', 'resource', 'efficiency', 'minimisation', 'missions', 'social', 'engagement', 'cience', 'health', 'nowledge', 'society', 'environment', 'introduction', 'esponsibility', 'utlook', 'appendix', 'integrity', 'iv', 'environment', 'community', 'ii', 'patients', 'iii', 'employees', 'five', 'pillars', 'table', 'contents', 'vifor', 'pharma', 'ltd', 'responsibility', 'highlights', 'introduction', 'commitment', 'vifor', 'pharma', 'constantly', 'evolving', 'seek', 'achieve', 'vision', 'global', 'leader', 'iron'

In [5]:
id2word = corpora.Dictionary(data)

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data]
print(corpus[0][:150])

[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 5), (8, 1), (9, 2), (10, 43), (11, 1), (12, 1), (13, 3), (14, 4), (15, 3), (16, 1), (17, 1), (18, 1), (19, 4), (20, 2), (21, 1), (22, 10), (23, 4), (24, 5), (25, 2), (26, 8), (27, 16), (28, 2), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 5), (37, 1), (38, 1), (39, 1), (40, 1), (41, 3), (42, 2), (43, 2), (44, 2), (45, 1), (46, 2), (47, 1), (48, 1), (49, 2), (50, 1), (51, 2), (52, 8), (53, 1), (54, 1), (55, 1), (56, 1), (57, 6), (58, 2), (59, 1), (60, 1), (61, 2), (62, 1), (63, 2), (64, 3), (65, 1), (66, 8), (67, 1), (68, 9), (69, 1), (70, 7), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 2), (77, 1), (78, 1), (79, 4), (80, 4), (81, 3), (82, 31), (83, 2), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 3), (90, 1), (91, 1), (92, 4), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 2), (99, 1), (100, 1), (101, 1), (102, 1), (103, 2), (104, 4), (105, 5), (106, 1), (107, 13), (108, 2), (109, 1), (110

In [6]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [7]:
pprint(lda_model.print_topics())
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

coherence_model_lda = CoherenceModel(model=lda_model, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


[(0,
  '0.000*"year" + 0.000*"board" + 0.000*"business" + 0.000*"data" + '
  '0.000*"company" + 0.000*"norwegian" + 0.000*"management" + 0.000*"new" + '
  '0.000*"directors" + 0.000*"remuneration"'),
 (1,
  '0.000*"year" + 0.000*"data" + 0.000*"norwegian" + 0.000*"company" + '
  '0.000*"management" + 0.000*"board" + 0.000*"business" + 0.000*"new" + '
  '0.000*"december" + 0.000*"pharma"'),
 (2,
  '0.001*"year" + 0.000*"executive" + 0.000*"business" + 0.000*"employees" + '
  '0.000*"board" + 0.000*"directors" + 0.000*"data" + 0.000*"committee" + '
  '0.000*"company" + 0.000*"new"'),
 (3,
  '0.019*"tonnes" + 0.009*"employee" + 0.007*"emissions" + 0.006*"sm" + '
  '0.004*"na" + 0.004*"sintef" + 0.004*"ntnu" + 0.004*"oslo" + '
  '0.002*"sustainability" + 0.002*"future"'),
 (4,
  '0.011*"year" + 0.007*"board" + 0.007*"directors" + 0.007*"data" + '
  '0.007*"committee" + 0.006*"annual" + 0.006*"december" + 0.006*"executive" + '
  '0.005*"new" + 0.005*"company"'),
 (5,
  '0.001*"year" + 0.001

In [8]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

pyLDAvis.display(vis)

/home/keval/envs/topicmodeling/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/home/keval/envs/topicmodeling/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/home/keval/envs/topicmodeling/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/home/keval/envs/topicmodeling/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for remov